Description This Code

- Sclaer : MinMax()

- Feature_Selection : Drop Soot , FNOX , FOXID, FSO4, ZN, V100

- XGB

(Optuna Tuning S-KFOLD
Best Trial: score 0.8292545137031471,<-m_F1
params {'max_depth': 10, 'learning_rate': 0.23720667234749193, 'n_estimators': 1100}

- LGB

hand-crafted paramter tuning
{'n_estimators':2000,'learning_rate':0.01, 'max_depth':13, 'n_jobs':-1,                 'boost_from_average':False}

- TH :0.23

-> 0.589653 (pub : 20)


#라이브러리 불러오기

In [ ]:
# 최적 파라미터를 범위내에서 찾아주는 라이브러리로써 캐글에서 인기 폭발이라고 한다.
!pip install optuna

In [ ]:
pip install xgboost

In [ ]:
pip install lightgbm

In [ ]:
pip install anaconda

In [ ]:
pip install catboost

- profiling_tool (굳이 안해도됨.)

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import gc

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'NanumGothic'
#한글폰트

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#구글드라이브연동

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#한글폰트 설치. 런타임재사용 필요

- 데이터불러오기. (경로설정해주기)

In [ ]:
train=pd.read_csv('/content/drive/MyDrive/DACON_Dataset/Hyundai_oil/train.csv')


In [ ]:
test=pd.read_csv('/content/drive/MyDrive/DACON_Dataset/Hyundai_oil/test.csv')

# dataframe 사이즈 줄이기.

In [ ]:
int64_col=list(train.columns[train.dtypes=='int64'])
float64_col=list(train.columns[train.dtypes=='float64'])

In [ ]:
# train_data size 줄이기. int64->int32
for i in int64_col :
  train[i]=train[i].astype('int32')
  

for i in float64_col : #float64 -> float32
  train[i]=train[i].astype('float32')
  

In [ ]:
int64_col=list(test.columns[test.dtypes=='int64'])
float64_col=list(test.columns[test.dtypes=='float64'])

In [ ]:
# test_data size 줄이기.
for i in int64_col :
  
  test[i]=test[i].astype('int32')

for i in float64_col :
  
  test[i]=test[i].astype('float32')

# 문자열 간단히 라벨링.

- 라벨인코더 사용. (ordinal_Encoder사용해도됨)

In [ ]:
cate_val = ['COMPONENT_ARBITRARY']
for i in cate_val :
  enc = LabelEncoder()
  enc=enc.fit(train[i])
  train[i]=enc.transform(train[i])
  for label in np.unique(test[i]):
        if label not in enc.classes_: 
            enc.classes_ = np.append(enc.classes_, label)
  test[i]=enc.transform(test[i]) # data leakage 주의

In [ ]:
train.COMPONENT_ARBITRARY.unique()

#NAN 확인 (test셋에는 결측치가 없다.)

In [ ]:
train.isnull().sum()

# exp1) 결측치가 한컴포넌트를 제외한 모든 값인경우 특성을 제외시켜보기.

In [ ]:
exclude_col=train.columns[train.isnull().sum()> 10000]

train=train.drop(exclude_col,axis=1)


In [ ]:
train.columns

#exp1) 공선성제거하기.
(S FE 제거해보기.)

In [ ]:
temp1=temp.drop(['S','FE'],axis=1)

In [ ]:
temp.columns

# Component별 데이터 분리.

In [ ]:
comp0 = train[train['COMPONENT_ARBITRARY']==0]
comp1 = train[train['COMPONENT_ARBITRARY']==1]
comp2 = train[train['COMPONENT_ARBITRARY']==2]
comp3 = train[train['COMPONENT_ARBITRARY']==3]

print(comp0.shape, comp1.shape, comp2.shape,comp3.shape)

#EDA

In [ ]:
train=temp

In [ ]:
comp0[['ANONYMOUS_1', 'YEAR', 'SAMPLE_TRANSFER_DAY',
       'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE']].plot(kind='box', subplots=True, layout=(3, 3), figsize=(20, 10))
plt.show()  

In [ ]:
def hist_draw(f_name,comp0):

  fig,axes=plt.subplots(1,2)
  plt.figure(figsize=(10,10))
  sns.histplot(comp0[comp0.Y_LABEL==0][f_name],ax=axes[0],bins=100,color='blue',stat='probability')
  sns.histplot(comp0[comp0.Y_LABEL==1][f_name],ax=axes[1],bins=100,color='red',stat='probability')

  #axes[0].set_xlim(0,2000)
  axes[0].set_title(f_name+'_normal')
  #axes[1].set_xlim(0,2000)
  axes[1].set_title( f_name+'_abnormal')


In [ ]:
hist_draw('PQINDEX',comp1)

- 컴포넌트별로 분포를 띄어보자.

In [ ]:
train.COMPONENT_ARBITRARY.unique()

In [ ]:
fig, ax = plt.subplots()
sns.histplot(comp0.ANONYMOUS_1, ax=ax,kde=True,color='blue',bins=300) 
sns.histplot(comp1.ANONYMOUS_1, ax=ax,kde=True,color='green',bins=300)
sns.histplot(comp2.ANONYMOUS_1, ax=ax,kde=True,color='red',bins=300)
sns.histplot(comp3.ANONYMOUS_1, ax=ax,kde=True,color='yellow',bins=300)
ax.set_xlim(0,50000)
ax.set_title('ANONYMOUS_1')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.ANONYMOUS_2, ax=ax[0],kde=True,color='blue',bins=50,stat='probability') 
sns.histplot(comp1.ANONYMOUS_2, ax=ax[1],kde=True,color='green',bins=50,stat='probability')
sns.histplot(comp2.ANONYMOUS_2, ax=ax[2],kde=True,color='red',bins=50,stat='probability')
sns.histplot(comp3.ANONYMOUS_2, ax=ax[3],kde=True,color='yellow',bins=50,stat='probability')

ax[0].set_title('ANONYMOUS_2')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.AG, ax=ax[0],kde=True,color='blue',bins=50,stat='probability') 
sns.histplot(comp1.AG, ax=ax[1],kde=True,color='green',bins=50,stat='probability')
sns.histplot(comp2.AG, ax=ax[2],kde=True,color='red',bins=50,stat='probability')
sns.histplot(comp3.AG, ax=ax[3],kde=True,color='yellow',bins=50,stat='probability')

ax[0].set_title('AG')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.CO, ax=ax[0],kde=True,color='blue',bins=50,stat='probability') 
sns.histplot(comp1.CO, ax=ax[1],kde=True,color='green',bins=50,stat='probability')
sns.histplot(comp2.CO, ax=ax[2],kde=True,color='red',bins=50,stat='probability')
sns.histplot(comp3.CO, ax=ax[3],kde=True,color='yellow',bins=50,stat='probability')

ax[0].set_title('CO')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.CR, ax=ax[0],kde=True,color='blue',bins=100,stat='probability') 
sns.histplot(comp1.CR, ax=ax[1],kde=True,color='green',bins=100,stat='probability')
sns.histplot(comp2.CR, ax=ax[2],kde=True,color='red',bins=100,stat='probability')
sns.histplot(comp3.CR, ax=ax[3],kde=True,color='yellow',bins=100,stat='probability')

ax[0].set_title('CR')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
comp3.CR.value_counts()



In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.CU, ax=ax[0],kde=True,color='blue',bins=100,stat='probability') 
sns.histplot(comp1.CU, ax=ax[1],kde=True,color='green',bins=100,stat='probability')
sns.histplot(comp2.CU, ax=ax[2],kde=True,color='red',bins=100,stat='probability')
sns.histplot(comp3.CU, ax=ax[3],kde=True,color='yellow',bins=100,stat='probability')

ax[0].set_title('CU')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.FOXID, ax=ax[0],kde=True,color='blue',bins=100,stat='probability') 
sns.histplot(comp1.FOXID, ax=ax[1],kde=True,color='green',bins=100,stat='probability')
sns.histplot(comp2.FOXID, ax=ax[2],kde=True,color='red',bins=100,stat='probability')
sns.histplot(comp3.FOXID, ax=ax[3],kde=True,color='yellow',bins=100,stat='probability')

ax[0].set_title('FOXID')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.U100, ax=ax[0],kde=True,color='blue',bins=50,stat='probability') 
sns.histplot(comp1.U100, ax=ax[1],kde=True,color='green',bins=50,stat='probability')
sns.histplot(comp2.U100, ax=ax[2],kde=True,color='red',bins=50,stat='probability')
sns.histplot(comp3.U100, ax=ax[3],kde=True,color='yellow',bins=50,stat='probability')

ax[0].set_title('U100')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.U14, ax=ax[0],kde=True,color='blue',bins=50,stat='probability') 
sns.histplot(comp1.U14, ax=ax[1],kde=True,color='green',bins=50,stat='probability')
sns.histplot(comp2.U14, ax=ax[2],kde=True,color='red',bins=50,stat='probability')
sns.histplot(comp3.U14, ax=ax[3],kde=True,color='yellow',bins=50,stat='probability')

ax[0].set_title('U14')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
comp3.CD.isnull().sum()

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(60,20))
sns.histplot(comp0.CD, ax=ax[0],kde=True,color='blue',bins=100,stat='probability') 
sns.histplot(comp1.CD, ax=ax[1],kde=True,color='green',bins=100,stat='probability')
sns.histplot(comp2.CD, ax=ax[2],kde=True,color='red',bins=100,stat='probability')
sns.histplot(comp3.CD, ax=ax[3],kde=True,color='yellow',bins=100,stat='probability')

ax[0].set_title('CD')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

In [ ]:
fig, ax = plt.subplots(4,1)
plt.figure(figsize=(100,40))
sns.histplot(data=comp0,x='V40', ax=ax[0],kde=True,hue='Y_LABEL',bins=50,stat='probability')
sns.histplot(data=comp1,x='V40', ax=ax[1],kde=True,hue='Y_LABEL',bins=50,stat='probability')
sns.histplot(data=comp2,x='V40', ax=ax[2],kde=True,hue='Y_LABEL',bins=50,stat='probability')
sns.histplot(data=comp3,x='V40', ax=ax[3],kde=True,hue='Y_LABEL',bins=50,stat='probability')

ax[0].set_title('V40')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

각 컴포넌트별 사이즈 :

0(3890, 54) 1(2316, 54) 2(7050, 54) 3(839, 54)


FH2O                   10205-> 10205는 0번을 제외한 모든 컴포넌트

FNOX                   10205


FOPTIMETHGLY           10205


FOXID                  10205
FSO4                   10205


FTBN                   10205


FUEL                   10205

SOOTPERCENTAGE         10205


-> component 0을 제외한 나머지 component는 수집되지 않거나, 측정할 수 없는 지표로 추정.

0(3890, 54) 1(2316, 54) 2(7050, 54) 3(839, 54)


U100                   11779->1번 컴포넌트를 제외한 모든 컴포넌트


U75                    11779


U50                    11779


U25                    11779


U20                    11779


->11977은 컴포넌트 1에서의 결측값이다.
U14                    11977->(198개는 컴포넌트1안에서의 결측값이다.)


U6                     11977


U4                     11977



컴포넌트 0을 제외한 모든 컴포넌트가 결측이며, + 컴포넌트0에도 결측치가 미세하게 존재한다.

V100                   10371


그 외 모든 결측열 ( K, CD)는 모든 컴포넌트에 골고루 존재함.

In [ ]:
fig, ax = plt.subplots(8,1)
plt.figure(figsize=(100,20))

sns.histplot(comp1[comp1.Y_LABEL==0].U4, ax=ax[0],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U4, ax=ax[0],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U6, ax=ax[1],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U6, ax=ax[1],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U14, ax=ax[2],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U14, ax=ax[2],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U20, ax=ax[3],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U20, ax=ax[3],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U25, ax=ax[4],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U25, ax=ax[4],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U50, ax=ax[5],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U50, ax=ax[5],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U75, ax=ax[6],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U75, ax=ax[6],kde=True,color='red',bins=50)

sns.histplot(comp1[comp1.Y_LABEL==0].U100, ax=ax[7],kde=True,color='green',bins=50)
sns.histplot(comp1[comp1.Y_LABEL==1].U100, ax=ax[7],kde=True,color='red',bins=50)


ax[0].set_title('U')
#ax.set_xlim(0,10000)
#ax.set_title('ANONYMOUS_2')

# 컴포넌트별 특성별 박스플롯.

In [ ]:
def comp_boxplot(comp_list,f_name):
  fig, ax = plt.subplots(2,4)
  plt.figure(figsize=(200,300))
  for i,col in zip(range(0,4,1),['blue','green','red','yellow']):
    
    
    sns.boxplot(data=comp_list[i],y=f_name, ax=ax[0][i],color=col)

    ax[0][i].set_title('comp'+str(i)+'normal') 
  for i,col in zip(range(0,4,1),['blue','green','red','yellow']):

    
    
    sns.boxplot(data=comp_list[i],y=f_name, ax=ax[1][i],color=col)

    ax[1][i].set_title('comp'+str(i)+'abnormal') 




In [ ]:
comp_list = [comp0,comp1,comp2,comp3]

comp_boxplot(comp_list,'ANONYMOUS_2')

In [ ]:
comp_boxplot(comp_list,'B')

In [ ]:
comp_boxplot(comp_list,'AL')

- 전체데이터셋에대한 eda

In [ ]:
sns.barplot(train.YEAR,train.Y_LABEL)

In [ ]:
sns.barplot(train.COMPONENT_ARBITRARY,train.Y_LABEL)

In [ ]:
sns.boxplot(y=train.CD,manage_xticks=False)

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.ANONYMOUS_1, ax=ax,kde=True,color='blue',bins=300,stat='probability')  # distplot is deprecate and replaced by histplot
sns.histplot(train[train.Y_LABEL==1]['ANONYMOUS_1'],color='red', ax=ax,kde=True,bins=150,stat='probability')
ax.set_xlim(0,50000)
ax.set_title('ANONYMOUS_1')
#ax.set_xticks(range(1,32))

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.ANONYMOUS_2, ax=ax,kde=True,color='blue',bins=10)  # distplot is deprecate and replaced by histplot
sns.histplot(train[train.Y_LABEL==1]['ANONYMOUS_2'],color='red', ax=ax,kde=True,bins=10)
ax.set_xlim(0,2000)
#ax.set_ylim(0,0.2)

ax.set_title('ANONYMOUS_2')
#ax.set_xticks(range(1,32))

- 비슷한 분포를 보이는 것으로 보아 별의미 없는 컬럼.

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.SAMPLE_TRANSFER_DAY,color='blue', ax=ax,kde=True,bins=150,stat='probability')
sns.histplot(train[train.Y_LABEL==1]['SAMPLE_TRANSFER_DAY'],color='red', ax=ax,kde=True,bins=150,stat='probability')  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,50)
ax.set_title('SAMPLE_TRANSFER_DAY')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.FH2O,color='blue', ax=ax,kde=True,bins=150,stat='probability')
sns.histplot(train[train.Y_LABEL==1]['FH2O'],color='red', ax=ax,kde=True,bins=150,stat='probability')  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,50)
ax.set_title('FH2O')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.FNOX,color='blue', ax=ax,kde=True,bins=50,stat='probability')
sns.histplot(train[train.Y_LABEL==1]['FNOX'],color='red', ax=ax,kde=True,bins=50,stat='probability')  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,50)
ax.set_title('FNOX')

- 최근에 점검했을수록 불량률 떨어진다.

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train[train.Y_LABEL==0]['YEAR'], ax=ax,color='blue',kde=True)  
sns.histplot(train[train.Y_LABEL==1]['YEAR'],color='red', ax=ax,kde=True)  
# distplot is deprecate and replaced by histplot

ax.set_title('YEAR')

- CD는 수치가 낮으면 불량일 가능성 존재.

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.CD, ax=ax,color='blue',kde=True,bins=300)  
sns.histplot(train[train.Y_LABEL==1]['CD'],color='red', ax=ax,kde=True,bins=300)  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,1.0)
ax.set_ylim(0,5000)

ax.set_title('CD')

In [ ]:
temp.columns

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train.AL, ax=ax,color='blue',kde=True,bins=1000)  
sns.histplot(train[train.Y_LABEL==1]['AL'],color='red', ax=ax,kde=True,bins=1000,stat='probability')  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,10)
#ax.set_ylim(0,5000)

ax.set_title('AL')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train['H2O'], ax=ax,color='blue',kde=True,bins=100,stat='probability')  
sns.histplot(train[train.Y_LABEL==1]['H2O'],color='red', ax=ax,kde=True,bins=100,stat='probability')  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,10)
ax.set_ylim(0,0.2)

ax.set_title('H2O')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(temp['CR'], ax=ax,color='blue',kde=True,bins=10,stat='probability')  
sns.histplot(temp[temp.Y_LABEL==1]['CR'],color='red', ax=ax,kde=True,bins=10,stat='probability')  
# distplot is deprecate and replaced by histplot
ax.set_xlim(0,500)
ax.set_ylim(0,5.0)

ax.set_title('CR')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(np.log(temp['AG']+0.0001), ax=ax,color='blue',kde=True,bins=10,stat='probability')  
sns.histplot(np.log(temp[temp.Y_LABEL==1]['AG']+0.0001),color='red', ax=ax,kde=True,bins=10,stat='probability')  
# distplot is deprecate and replaced by histplot
#ax.set_xlim(0,500)
#ax.set_ylim(0,5.0)

ax.set_title('AG')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(comp0[comp0.Y_LABEL==0]['SOOTPERCENTAGE'], ax=ax,color='blue',kde=True)  
sns.histplot(comp0[comp0.Y_LABEL==1]['SOOTPERCENTAGE'],color='red', ax=ax,kde=True)  
# distplot is deprecate and replaced by histplot

ax.set_title('SOOTPERCENTAGE')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train[train.Y_LABEL==0]['V40'], ax=ax,color='blue',kde=True)  
sns.histplot(train[train.Y_LABEL==1]['V40'],color='red', ax=ax,kde=True)  
# distplot is deprecate and replaced by histplot

ax.set_xlim(0,500)

ax.set_title('V40')

In [ ]:
fig, ax = plt.subplots()
sns.histplot(train[train.Y_LABEL==0]['V100'], ax=ax,color='blue',kde=True)  
sns.histplot(train[train.Y_LABEL==1]['V100'],color='red', ax=ax,kde=True)  
# distplot is deprecate and replaced by histplot

ax.set_xlim(0,60)

ax.set_title('V100')

In [ ]:
train.BE.plot()

- 컬럼별 왜도확인

In [ ]:
int32_col=list(train.columns[train.dtypes=='int32'])
float32_col=list(train.columns[train.dtypes=='float32'])

In [ ]:
num_val=int32_col +float32_col

In [ ]:
tmp = ['YEAR','Y_LABEL']
num_val = [x for x in num_val if x not in tmp]

In [ ]:
test_num_col=list(test.columns[(test.columns !='YEAR') & (test.columns !='COMPONENT_ARBITRARY') ])

In [ ]:
from scipy.stats import skew, kurtosis,boxcox

skew_num_val=[]
for i in test_num_col:
  
  print(i," : ",skew((temp[i])))
  if abs(skew(temp[i]))>1:

    skew_num_val.append(i)



- 상관계수 분석 (다중공선성 제거를 위한)

In [ ]:
train.corr(method='pearson')

In [ ]:
train.corr()

In [ ]:
test.corr()

In [ ]:
temp1=train.copy()

In [ ]:
temp1.columns

In [ ]:
train.columns

num_val = [x for x in train.columns if x not in temp1.columns]

In [ ]:
num_val

# Corr

In [ ]:
temp

In [ ]:
temp1 =temp.copy()

In [ ]:
data = temp.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(100,100) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(data, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(data, 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )
plt.title('0.5<coef_variables_onTest\n',fontsize=30)  
plt.show()

In [ ]:
temp.columns

In [ ]:
temp.drop(['V100','ZN'],axis=1,inplace=True)

In [ ]:
temp.drop('S',axis=1,inplace=True)

In [ ]:
temp.drop('V',axis=1,inplace=True) #,'S'

In [ ]:
test.drop(['ZN'],axis=1,inplace=True)

In [ ]:
test.drop(['V'],axis=1,inplace=True)

In [ ]:
temp.drop('MN',axis=1,inplace=True)

In [ ]:
test.drop(['MN'],axis=1,inplace=True)

In [ ]:
temp.drop(['SOOTPERCENTAGE'],axis=1,inplace=True)

In [ ]:
temp.drop('FNOX',axis=1,inplace=True)

In [ ]:
temp.drop('FOXID',axis=1,inplace=True)

In [ ]:
temp.drop('FSO4',axis=1,inplace=True)

결측치가 없는 Test셋에서는.

 - MN & FE ( 0.72)
 - NI & FE ( 0.59)
 - NI & CU ( 0.69)
 - NI & MN ( 0.54)
 - TI & MN ( 0.58)
 - V  & TI (0.73) 
 - FE & CO ( 0.59)


In [ ]:
data = train[test.columns].corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(15,15) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(data, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(data[abs(data)>0.5], 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )
plt.title('0.5<coef_variables_onTrain(Test_col)\n',fontsize=30)  
plt.show()

결측치가 있는 Train셋에서는.

 - MN & FE ( 0.62)
 - NI & FE ( 0.58)

 - NI & MN ( 0.53)
 - TI & MN ( 0.59)
 - V  & TI (0.68) 
 - FE & CO ( 0.54)

 

---
NI & CU 높은 상관관계가 사라짐.

-> 결측치를 제거 (iterative or knn) 한 후에
test분포와 비교해봄으로써, 합리적인 대체인지 확인한다. 


In [ ]:
data = train.corr()
# 그림 사이즈 지정
fig, ax = plt.subplots( figsize=(30,30) )

# 삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래 삼각형에 False)
mask = np.zeros_like(data, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# 히트맵을 그린다
sns.heatmap(data[abs(data)>0.5], 
            cmap = 'RdYlBu_r', 
            annot = True,   # 실제 값을 표시한다
            
            mask=mask,      # 표시하지 않을 마스크 부분을 지정한다
            linewidths=.5,  # 경계면 실선으로 구분하기
            cbar_kws={"shrink": .5},# 컬러바 크기 절반으로 줄이기
            vmin = -1,vmax = 1   # 컬러바 범위 -1 ~ 1
           )
plt.title('0.5<coef_variables_onTrain(ALL)\n',fontsize=30)  
plt.show()

In [ ]:
print(train[train['COMPONENT_ARBITRARY']==3].isnull().sum())
train_c0=train[train['COMPONENT_ARBITRARY']==3]
print(train_c0.shape)

In [ ]:
train_c0[train_c0['Y_LABEL']==1].isnull().sum()

In [ ]:
train_c0.FH2O

- Isolation_Forest를 통한 이상치 탐지.

In [ ]:
comp2

In [ ]:
comp2

# 결측치 처리하기.

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
train.describe()

- 간단한 샘플 결과제출을 위한 knn_imputer적용 

In [ ]:
train=train.drop('ID',axis=1) # 필요없는 ID제거.
test=test.drop('ID',axis=1)

- 좀더 합리적인 결측치처리가 필요함. 

In [ ]:
comp0=comp0.drop('ID',axis=1)
comp1=comp1.drop('ID',axis=1)
comp2=comp2.drop('ID',axis=1)
comp3=comp3.drop('ID',axis=1)

- knn_imputer

In [ ]:
zero_fill = ['FH2O','FNOX','FOXID','FSO4','FOPTIMETHGLY','FTBN','FUEL','SOOTPERCENTAGE','U100','U75','U50','U25','U20']
for i in zero_fill:
  
  comp0[i]=comp0[i].fillna(0)
  comp1[i]=comp1[i].fillna(0)
  comp2[i]=comp2[i].fillna(0)
  comp3[i]=comp3[i].fillna(0)


In [ ]:
zero_fill1=['U14','U6','U4']

for i in zero_fill1:
   comp0[i]=comp0[i].fillna(0)
   comp2[i]=comp2[i].fillna(0)
   comp3[i]=comp3[i].fillna(0)

  

In [ ]:
comp1['V100']=comp1['V100'].fillna(0)
comp2['V100']=comp2['V100'].fillna(0)
comp3['V100']=comp3['V100'].fillna(0)

In [ ]:
train.columns

In [ ]:


imputer = KNNImputer(n_neighbors=2)
data_imputed=imputer.fit_transform(comp0)
comp0=pd.DataFrame(data_imputed)
comp0.columns=train.columns

imputer = KNNImputer(n_neighbors=2)
data_imputed=imputer.fit_transform(comp1)
comp1=pd.DataFrame(data_imputed)
comp1.columns=train.columns

imputer = KNNImputer(n_neighbors=2)
data_imputed=imputer.fit_transform(comp2)
comp2=pd.DataFrame(data_imputed)
comp2.columns=train.columns

imputer = KNNImputer(n_neighbors=2)
data_imputed=imputer.fit_transform(comp3)
comp3=pd.DataFrame(data_imputed)
comp3.columns=train.columns


temp=pd.concat([comp0,comp1,comp2,comp3])
temp

In [ ]:
temp.isnull().sum()

- YEAR , COMPONENT_ARBITRARY, Y_LABEL 
INT형으로 변환

In [ ]:
temp[['YEAR','COMPONENT_ARBITRARY','Y_LABEL']]=temp[['YEAR','COMPONENT_ARBITRARY','Y_LABEL']].astype('int')

In [ ]:
test[['YEAR','COMPONENT_ARBITRARY']]=test[['YEAR','COMPONENT_ARBITRARY']].astype('int')

- simple_imputer (중앙값 대치)

In [ ]:
from sklearn.impute import SimpleImputer

# 중간값으로 Imputer 선언
imputer_mid = SimpleImputer(strategy='median')
imputer_mid.fit(train)
data_imputed=imputer_mid.transform(train)
temp=pd.DataFrame(data_imputed)

- iterative_imputer

In [ ]:
imputer_it = IterativeImputer(max_iter = 30, random_state = 0)
data_imputed=imputer_it.fit_transform(train)
temp=pd.DataFrame(data_imputed)

# Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
comp_list= [ comp0,comp1,comp2,comp3]
c_n0=comp0[comp0.Y_LABEL==0]
c_n1=comp1[comp1.Y_LABEL==0]
c_n2=comp2[comp2.Y_LABEL==0]
c_n3=comp3[comp3.Y_LABEL==0]

c_an0=comp0[comp0.Y_LABEL==1]
c_an1=comp1[comp1.Y_LABEL==1]
c_an2=comp2[comp2.Y_LABEL==1]
c_an3=comp3[comp3.Y_LABEL==1]

comp_normal =[c_n0,c_n1,c_n2,c_n3]
comp_abnormal =[c_an0,c_an1,c_an2,c_an3]


- 정상데이터 이상치 제거.

In [ ]:
for com in comp_normal:
  model=IsolationForest(n_estimators=400, max_samples='auto', n_jobs=-1,
                      max_features=2, contamination=0.01,random_state=6)
  model.fit(com.to_numpy())
  score = model.decision_function(com.to_numpy())
  anomaly = model.predict(com.to_numpy())
  
  com['anomaly']= anomaly
  anomaly_data = com.loc[com['anomaly']==-1] # 이상값은 -1으로 나타낸다.
  anomaly_data

In [ ]:
c_n0.columns

In [ ]:
c_n0=c_n0.drop(index=c_n0[c_n0['anomaly']==-1].index,axis=0)
c_n1=c_n1.drop(index=c_n1[c_n1['anomaly']==-1].index,axis=0)
c_n2=c_n2.drop(index=c_n2[c_n2['anomaly']==-1].index,axis=0)
c_n3=c_n3.drop(index=c_n3[c_n3['anomaly']==-1].index,axis=0)

In [ ]:
c_n0=c_n0.drop('anomaly',axis=1)
c_n1=c_n1.drop('anomaly',axis=1)
c_n2=c_n2.drop('anomaly',axis=1)
c_n3=c_n3.drop('anomaly',axis=1)

- 비정상데이터 이상치제거.

In [ ]:
for com in comp_abnormal:


  model=IsolationForest(n_estimators=400, max_samples='auto', n_jobs=-1,
                      max_features=2, contamination=0.01,random_state=3)
  model.fit(com.to_numpy())
  score = model.decision_function(com.to_numpy())
  anomaly = model.predict(com.to_numpy())
  
  com['anomaly']= anomaly
  anomaly_data = com.loc[com['anomaly']==-1] # 이상값은 -1으로 나타낸다.
  anomaly_data

In [ ]:
c_an0=c_an0.drop(index=c_an0[c_an0['anomaly']==-1].index,axis=0)
c_an1=c_an1.drop(index=c_an1[c_an1['anomaly']==-1].index,axis=0)
c_an2=c_an2.drop(index=c_an2[c_an2['anomaly']==-1].index,axis=0)
c_an3=c_an3.drop(index=c_an3[c_an3['anomaly']==-1].index,axis=0)

In [ ]:
c_an0=c_an0.drop('anomaly',axis=1)
c_an1=c_an1.drop('anomaly',axis=1)
c_an2=c_an2.drop('anomaly',axis=1)
c_an3=c_an3.drop('anomaly',axis=1)

In [ ]:
temp=pd.concat([c_n0,c_an0,c_n1,c_an1,c_n2,c_an2,c_n3,c_an3],axis=0)

In [ ]:
temp.isnull().sum()

In [ ]:
temp.shape

# Simple_profile ( x)

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(train)

In [ ]:
profile.to_notebook_iframe()

In [ ]:
profile.to_file(output_file="train_profiling.html") # html

# U(입자크기관련 컬럼들 범위조정을 통한 공선성 없애기.)

In [ ]:
comp1.U4=comp1.U4-comp1.U6
comp1.U6=comp1.U6-comp1.U14
comp1.U14=comp1.U14-comp1.U20
comp1.U20=comp1.U20-comp1.U25
comp1.U25=comp1.U25-comp1.U50
comp1.U50=comp1.U50-comp1.U75
comp1.U75=comp1.U75-comp1.U100

#Scaling ( MinMax Scaler | Quantile| Power(BOXCOX)

- test데이터의 18개 컬럼에대한 스케일링을 먼저 진행

  - YEAR , COMPONENT_ARBITRARY는 스케일링 컬럼에서 제외시켜주기.

In [ ]:
temp.columns

In [ ]:
test_num_col=list(test.columns[(test.columns !='YEAR') & (test.columns !='COMPONENT_ARBITRARY') ])

- train데이터의 51개 컬럼중 num컬럼만 가져오기. ( Y_LABEL , YEAR 제외시켜주기.)

In [ ]:
int32_col=list(train.columns[train.dtypes=='int32'])
float32_col=list(train.columns[train.dtypes=='float32'])

In [ ]:
num_val=int32_col +float32_col

In [ ]:
train_num_col = [x for x in num_val if x not in test_num_col]


In [ ]:
tmp = ['YEAR','Y_LABEL','SAMPLE_TRANSFER_DAY']
train_num_col = [x for x in num_val if x not in tmp]

-student test셋에 해당하는 num컬럼에 대해서 먼저 스케일링진행하기.

In [ ]:
#int64_col float64_col
# test에 해당하는 num열 스케일링.
#here
from sklearn.preprocessing import MinMaxScaler, PowerTransformer,StandardScaler

min_max=test_num_col
train_num = temp[min_max].copy()

test_num = test[min_max].copy()


scaler=MinMaxScaler()

scaler.fit(train_num)

train_num[min_max]=scaler.transform(train_num[min_max])



test_num[min_max]=scaler.transform(test_num[min_max]) # leakage 주의


temp[min_max] = train_num[min_max]
test[min_max] = test_num[min_max]  

In [ ]:
for i in test_num_col:
  print(i," : ",skew(temp[i]))
  print("test"," : ",skew(test[i]))

- Qunantile_transformer ( MinMax or Quanitle 중 고르는 것.)

In [ ]:
#int64_col float64_col
# test에 해당하는 num열 스케일링.

from sklearn.preprocessing import MinMaxScaler, StandardScaler,QuantileTransformer

min_max=test_num_col
train_num = temp[min_max].copy()

test_num = test[min_max].copy()


scaler=QuantileTransformer(n_quantiles=1000)

scaler.fit(train_num)

train_num[min_max]=scaler.transform(train_num[min_max])



test_num[min_max]=scaler.transform(test_num[min_max]) # leakage 주의


temp[min_max] = train_num[min_max]
test[min_max] = test_num[min_max]  

-teacher train셋에 해당하는 num컬럼에 대해서 스케일링진행하기.

In [ ]:
from sklearn.preprocessing import RobustScaler,QuantileTransformer, StandardScaler
#here
min_max=train_num_col
train_num = temp[min_max].copy()

scaler=MinMaxScaler()

scaler.fit(train_num)

train_num[min_max]=scaler.transform(train_num[min_max])


temp[min_max] = train_num[min_max]


- Qunantile_transformer

In [ ]:
#int64_col float64_col
# teacher_train에 해당하는 num열 스케일링.

from sklearn.preprocessing import MinMaxScaler, StandardScaler,QuantileTransformer

min_max=train_num_col
train_num = temp[min_max].copy()

scaler=QuantileTransformer(n_quantiles=1000) # 분위수는 조절가능.

scaler.fit(train_num)


train_num[min_max]=scaler.transform(train_num[min_max])


temp[min_max] = train_num[min_max]

In [ ]:
for i in train_num_col:
  print(i," : ",skew(temp[i]))
 

In [ ]:
temp.SAMPLE_TRANSFER_DAY = np.log(temp.SAMPLE_TRANSFER_DAY)

#Feature_Selection (model_based)

- 랜덤포레스트를 통한 특성중요도를 뽑아 넘파이 행렬 형태로 저장. (성능향상 x)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(
    RandomForestClassifier(n_estimators=700,random_state=42),threshold='mean')


In [ ]:
y=temp['Y_LABEL']
temp=temp.drop('Y_LABEL',axis=1)
select.fit(temp,y)
f_temp=select.transform(temp)

f_temp.shape

#Feature_Selection (Based Corr)

In [ ]:
temp.drop(['V100','ZN'],axis=1,inplace=True)

In [ ]:
test.drop(['ZN'],axis=1,inplace=True)

In [ ]:
temp.drop(['SOOTPERCENTAGE'],axis=1,inplace=True)

In [ ]:
temp.drop('FNOX',axis=1,inplace=True)

In [ ]:
temp.drop('FOXID',axis=1,inplace=True)

In [ ]:
temp.drop('FSO4',axis=1,inplace=True)

In [ ]:
temp['U25low']= temp.U4 + temp.U6 + temp.U14+ temp.U25
temp['U25high']= temp.U25 + temp.U50 + temp.U75+ temp.U100

temp.drop(['U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4'],axis=1,inplace=True)

In [ ]:
temp.columns

# LGBM Classifier or XGBOOST ( choose one!)

In [ ]:
temp.columns

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score,roc_auc_score,roc_curve

In [ ]:
import lightgbm as lgb
import xgboost as xgb
#import catboost as cb

- train 인덱스 재정의

In [ ]:
temp=temp1

In [ ]:
temp=temp.reset_index(drop=True)

In [ ]:
temp1.columns

- valid_set .2로 설정. (Hold_Out으로 일단 확인)
- K-Fold를 통해 평균을 냄으로써, 과적합 및 일반화 성능을 올려야함.

In [ ]:
test.columns

In [ ]:
temp=temp.drop('SAMPLE_TRANSFER_DAY',axis=1)

- LGBM

- 카테고리컬 컬럼에 대해서 전처리.
(LGBM은 category 타입으로 지정해주면 categorical하게 처리한다.)

In [ ]:
temp[['YEAR','COMPONENT_ARBITRARY']]=temp[['YEAR','COMPONENT_ARBITRARY']].astype('category')

In [ ]:
test[['YEAR','COMPONENT_ARBITRARY']]=test[['YEAR','COMPONENT_ARBITRARY']].astype('category')

In [ ]:
temp.dtypes

- 아래쪽에 Optuna를 활용하여 파라미터튜닝을 간단히 할 수 있다. 

-> Optuna_Tuning(first_classification)

- xgboost와 lgbm마다 파라미터 튜닝이 다르기 때문에 params를 구글링을 통해 바꾸어서 실행할것.

- xgboost ( xgboost는 카테고리컬하게 처리하기 위해선, 원핫 인코딩이나 라벨링(int형변수)이여야함.

In [ ]:
temp[['COMPONENT_ARBITRARY', 'YEAR']] = temp[['COMPONENT_ARBITRARY', 'YEAR']].astype('int')
test[['COMPONENT_ARBITRARY', 'YEAR']] = test[['COMPONENT_ARBITRARY', 'YEAR']].astype('int')

In [ ]:
temp.Y_LABEL = temp.Y_LABEL.astype('int')

In [ ]:
y=temp['Y_LABEL']
temp=temp.drop('Y_LABEL',axis=1)
# train .8 valid .2
#x_train , x_valid , y_train, y_valid=train_test_split(temp,y,test_size=.2,stratify=y,random_state=5)

- 혼동행렬, 정확도(AC), 정밀도(PR), 재현율(RR), F1(MACRO), AUC 출력. 

In [ ]:
def get_clf_eval(y_test, y_pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    m_F1 = f1_score(y_test, y_pred,average='macro')
    AUC = roc_auc_score(y_test, pred_proba)

    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('MACRO_F1: {:.4f}'.format(m_F1))
    print('AUC: {:.4f}'.format(AUC))

- stratified_kfold 적용하기. (o)
1. fold 별 optuna를 최적 파라미터를 찾고 val-set의 평균을 낸다. 
2. fold별 최적파라미터를 이용해 train셋을 예측하고 평균을 구한다.





'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'SAMPLE_TRANSFER_DAY',
       'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA', 'CD', 'CO', 'CR',
       'CU', 'FH2O', 'FOPTIMETHGLY', 'FTBN', 'FE', 'FUEL', 'H2O', 'K', 'LI',
       'MG', 'MN', 'MO', 'NA', 'NI', 'P', 'PB', 'PQINDEX', 'S', 'SB', 'SI',
       'SN', 'TI', 'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V',
       'V40' 
       
       Drop) 'FNOX', 'FOXID', 'FSO4', 'ZN', 'V100' 'SOOTPERCENTAGE'

In [ ]:
from sklearn.model_selection import StratifiedKFold

# split 개수, 셔플 여부 및 seed 설정
model_history=[]
str_kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 50)
f_score=[]
train_proba=[]
train_pred=[]
# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in str_kf.split(temp,y):
    x_train, x_valid = temp.iloc[train_index], temp.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    optuna_cbrm = optuna.create_study(direction='maximize', sampler=sampler)
    optuna_cbrm.optimize(objective, n_trials=20)
    cbrm_trial = optuna_cbrm.best_trial
    cbrm_trial_params = cbrm_trial.params

    clf = xgb.XGBClassifier(**cbrm_trial.params,use_missing=False,boost_from_average=False)
    clf.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=200,verbose=True)

    # valid_set 성능 및 student_set 예측하기.
    val_pred=clf.predict(x_valid)
    val_proba=clf.predict_proba(x_valid)[:,1]
    f_score.append(f1_score(y_valid, val_pred))
    train_proba.append(clf.predict_proba(temp)[:,1])
    train_pred.append(clf.predict(temp))
    get_clf_eval(y_valid,val_pred,val_proba)

print('S-KFold f1score mean : ' ,np.mean(f_score))
train_p=pd.DataFrame(np.mean(train_proba,axis=0))
train_p


In [ ]:
for th in range(10,40,1):
  temp_sub = train_p.copy()
  example=pd.DataFrame(temp_sub)
  example.columns=['pred']
  example['pred'][example['pred']>(th*0.01)]=1.0
  example['pred'][example['pred']<=(th*0.01)]=0.0

  m_F1 = f1_score(y, example.pred,average='macro')
  print('th : {:.1f} MACRO_F1: {:.4f}'.format(th,m_F1))



In [ ]:
train_proba

- optuna_tuning 진행 후 실행할 것.

In [ ]:
0#here
clf = xgb.XGBClassifier(**cbrm_trial.params,use_missing=False,boost_from_average=False)
clf.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=60,verbose=True)

{'max_depth': 10, 'learning_rate': 0.23720667234749193, 'n_estimators': 1100}

- feature_importance

- 트리모델들은 특성중요도라는 항목을 제공.

In [ ]:
# 특성 중요도 뽑기.
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
plot_feature_importance(clf.feature_importances_,x_valid.columns,'XGBOOST')

- lgbm(category -> astype('categry')로 변경해주면됨.
위에 코드 존재.

- 실행x

In [ ]:

from sklearn.model_selection import StratifiedKFold

# split 개수, 셔플 여부 및 seed 설정
model_history=[]
val_history=[]
val_pred_hist=[]
str_kf = StratifiedKFold(n_splits = 5)
lgb_clf = lgb.LGBMClassifier(n_estimators=1000, boosting_type='gbdt',learning_rate=0.23,max_depth=8, n_jobs=-1,boost_from_average=False)
# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in str_kf.split(temp,y):
    X_train, X_test = temp.loc[train_index], temp.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    lgb_clf.fit(X_train,y_train)
    val_pred=lgb_clf.predict(X_test)
    val_pred_hist.append(val_pred)
    val_proba=lgb_clf.predict_proba(X_test)[:,1]
    val_history.append(val_proba)
    get_clf_eval(y_test,val_pred,val_proba)
    

In [ ]:
lgb_clf = lgb.LGBMClassifier(n_estimators=1000,eval_metric='AUC', boosting_type='gbdt',use_missing=False,learning_rate=0.04,max_depth=13, n_jobs=-1,boost_from_average=False)

In [ ]:
lgb_clf.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],verbose=True,early_stopping_rounds=200,eval_metric='logloss')

- valid_set으로 예측한 macro-f1 score
(xgb)


In [ ]:
#here
val_pred=clf.predict(x_valid)
val_proba=clf.predict_proba(x_valid)[:,1]
get_clf_eval(y_valid,val_pred,val_proba)


오차행렬:
 [[2597   12]
 [  96  114]]

정확도: 0.9617
정밀도: 0.9048
재현율: 0.5429
F1: 0.6786
MACRO_F1: 0.8291
AUC: 0.8867

- valid_set으로 예측한 macro-f1 score
(lgb)


In [ ]:
val_pred=lgb_clf.predict(x_valid)
val_proba=lgb_clf.predict_proba(x_valid)[:,1]
get_clf_eval(y_valid,val_pred,val_proba)

- train proba 예측 및 컬럼 생성.
    - xgb train_proba

In [ ]:
#here
val_pred=clf.predict(temp)
val_proba=clf.predict_proba(temp)[:,1]
get_clf_eval(y,val_pred,val_proba)

In [ ]:
temp.dtypes

- lgbm train_proba

In [ ]:
val_pred=lgb_clf.predict(temp)
val_proba=lgb_clf.predict_proba(temp)[:,1]
get_clf_eval(y,val_pred,val_proba)

In [ ]:
x_valid.dtypes

# 이젠 test_col X로 위에서 나온 확률 값을 예측하고, test set prediction 

- lgbm_regressor를 사용해서 test_col X로 proba예측


In [ ]:
test.columns

In [ ]:
test.drop('FE',axis=1,inplace=True)

In [ ]:
val_proba=train_p

In [ ]:
test_col=test.columns

In [ ]:
test_col=['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG', 'CO',
       'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40']

In [ ]:
temp_regress = temp[test_col]

In [ ]:
temp_regress['Y_PROBA']=val_proba

In [ ]:
temp_regress.Y_PROBA

In [ ]:
temp_regress.dtypes

In [ ]:
test

# Proba 예측을 위한 회귀 모형 생성.

- valid_set .2로 설정. (Hold_Out으로 일단 확인)

In [ ]:
y_regress=temp_regress['Y_PROBA']
temp_regress=temp_regress.drop('Y_PROBA',axis=1)

#Feature_Selection (model_based)

- 실행하지말것.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
select = SelectFromModel(
    RandomForestRegressor(n_estimators=1000,random_state=42),threshold='mean')


In [ ]:
#y=temp['Y_LABEL']
#temp=temp.drop('Y_LABEL',axis=1)
select.fit(temp_regress,y_regress)
f_temp=select.transform(temp_regress)
t_temp=select.transform(test)
f_temp.shape

# 회귀 모델링

In [ ]:
def best_th(y_valid, y_pred):

  m_f1=0.0
  for th in range(10,40,1):

    temp_sub = y_pred
    example=pd.DataFrame(temp_sub)
    example.columns=['pred']
    example['pred'][example['pred']>(th*0.01)]=1.0
    example['pred'][example['pred']<=(th*0.01)]=0.0

    if f1_score(y_valid, example.pred,average='macro') > m_f1:
      m_f1 =f1_score(y_valid, example.pred,average='macro')
    
  return m_f1
  


- Hold Out

In [ ]:
#.8 .2
x_r_train , x_r_valid , y_r_train, y_r_valid=train_test_split(temp_regress,y_regress,test_size=.2,random_state=5)

- kfold Optuna

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
# split 개수, 셔플 여부 및 seed 설정
model_history=[]
str_kf = KFold(n_splits = 10, shuffle = True, random_state = 50)
k_TH=[]
test_pred=[]
# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in str_kf.split(temp_regress,y_regress):


    x_r_train, x_r_valid = temp_regress.iloc[train_index], temp_regress.iloc[test_index]
    y_r_train, y_r_valid = y_regress.iloc[train_index], y_regress.iloc[test_index]
    # 성능확인을 위한 y_train값과 y_valid값을 가져오기.
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    optuna_cbrm_reg = optuna.create_study(direction='minimize', sampler=sampler)
    optuna_cbrm_reg.optimize(objective, n_trials=15)
    cbrm_trial_reg = optuna_cbrm_reg.best_trial
    cbrm_trial_params = cbrm_trial_reg.params

    clf = xgb.XGBRegressor(**cbrm_trial.params,use_missing=False,boost_from_average=False)
    clf.fit(x_r_train,y_r_train,eval_set=[(x_r_valid,y_r_valid)],early_stopping_rounds=70,verbose=True)

    # valid_set 성능 및 student_set 예측하기.
    val_pred=clf.predict(x_r_valid)
    k_TH.append(best_th(y_valid,val_pred))
    print(mean_absolute_error(y_r_valid,val_pred))
    test_pred.append(clf.predict(test))
    


test_p=pd.DataFrame(np.mean(test_pred,axis=0))
test_p




In [ ]:
k_TH

- lgb

In [ ]:
import lightgbm as lgb
#here
clf_r = lgb.LGBMRegressor(n_estimators=3000,learning_rate=0.01, max_depth=15, n_jobs=-1, 
                         boost_from_average=False)



- lgb_optuna

In [ ]:
import lightgbm as lgb

clf_r = lgb.LGBMRegressor(**trial_params,boost_from_average=False)


- xgb

In [ ]:

clf_x = xgb.XGBRegressor(**cbrm_trial.params,boost_from_average=False)



- 평가지표는 MeanAbsoluteError

- xgboost_regress

In [ ]:
clf_x.fit(x_r_train,y_r_train,eval_set=[(x_r_valid,y_r_valid)],verbose=True)

- lightgbm_regress

In [ ]:
clf_r.fit(x_r_train,y_r_train,eval_set=[(x_r_valid,y_r_valid)],verbose=True,eval_metric='mae')

- lightgbm_performance

In [ ]:
from sklearn.metrics import mean_absolute_error
val_r_prob=clf_r.predict(x_r_valid)
train_r_prob=clf_r.predict(x_r_train)
train_mae=mean_absolute_error(y_r_train,train_r_prob)
valid_mae=mean_absolute_error(y_r_valid,val_r_prob)
print('train_MAE: {:.4f}'.format(train_mae))
print('valid_MAE: {:.4f}'.format(valid_mae))


- xgboost_performance

In [ ]:
from sklearn.metrics import mean_absolute_error
val_r_prob=clf_x.predict(x_r_valid)
train_r_prob=clf_x.predict(x_r_train)
train_mae=mean_absolute_error(y_r_train,train_r_prob)
valid_mae=mean_absolute_error(y_r_valid,val_r_prob)
print('train_MAE: {:.4f}'.format(train_mae))
print('valid_MAE: {:.4f}'.format(valid_mae))


In [ ]:
x_r_valid.dtypes

In [ ]:
val_r_prob[val_r_prob>0.2].shape

train_MAE: 0.0986
valid_MAE: 0.1192

# TEST set 예측 생성

- 최적 TH을 일단 찾기.

In [ ]:
#train_mae=mean_absolute_error(y_r_train,train_r_prob)
for th in range(10,40,1):
  temp_sub = val_r_prob.copy()
  example=pd.DataFrame(temp_sub)
  example.columns=['pred']
  example['pred'][example['pred']>(th*0.01)]=1.0
  example['pred'][example['pred']<=(th*0.01)]=0.0

  m_F1 = f1_score(y_valid, example.pred,average='macro')
  print('th : {:.1f} MACRO_F1: {:.4f}'.format(th,m_F1))


In [ ]:
test_p=pd.DataFrame(np.mean(test_pred,axis=0))

- xgb_prediction

In [ ]:
sub_proba=clf_x.predict(test)

- lgbm_prediction

In [ ]:
sub_proba=clf_r.predict(test)

In [ ]:
sub_proba=test_p

In [ ]:
test_p

In [ ]:
example=pd.DataFrame(sub_proba)

example.columns=['pred']

In [ ]:
example

- th는 변환가능.

In [ ]:
example[example['pred']>0.20]

In [ ]:
example['pred'][example['pred']>0.195]=1.0
example['pred'][example['pred']<=0.195]=0.0

In [ ]:
example.pred.value_counts()

# 제출파일 형성.

In [ ]:
submit=pd.read_csv('/content/drive/MyDrive/DACON_Dataset/Hyundai_oil/sample_submission.csv')

In [ ]:
example.pred = example.pred.astype('int32')

In [ ]:
submit.Y_LABEL = example.pred

In [ ]:
submit.to_csv('/content/drive/MyDrive/DACON_Dataset/Hyundai_oil/sample19_sub.csv',index=False)

#Optuna_Tuning ( first_classification)

In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.model_selection import StratifiedKFold

In [ ]:
x_train.dtypes

    cbrm_param = {
        "objective": "binary:logistic",
        "eval_metric":'logloss',
        "booster": 'gbtree',
        #'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시
        "tree_method": 'gpu_hist', 'gpu_id': 0,  # CPU 사용시
        "verbosity": 0,
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.08, 0.3),
        'n_estimators': trial.suggest_int("n_estimators",100,1000 ,step=50),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 0.7),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.9, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),     
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True)
    }

Best Trial: score 0.8269007480183096,
params {'max_depth': 4, 'learning_rate': 0.23077916004091537, 'n_estimators': 1000}

In [ ]:
temp[['COMPONENT_ARBITRARY', 'YEAR']] = temp[['COMPONENT_ARBITRARY', 'YEAR']].astype('int')
test[['COMPONENT_ARBITRARY', 'YEAR']] = test[['COMPONENT_ARBITRARY', 'YEAR']].astype('int')

In [ ]:
# 데이터 분리에서 x_train , x_valid , y_train , y_valid 
#분리해놓은 것을 통해 최적 파라미터를 탐색한다.
#Best Trial: score 0.8174589706374902,
#params {'max_depth': 6, 'learning_rate': 0.0896457098432712, 'n_estimators': 800, 'colsample_bytree': 0.5569921809317748, 'colsample_bylevel': 0.9143406631538383, 'colsample_bynode': 0.5234481596946249, 'reg_lambda': 0.17888515294488294, 'reg_alpha': 0.12450084866814962, 'subsample': 0.95, 'min_child_weight': 4, 'gamma': 0.7192010329138374}
# random sampler

#Best Trial: score 0.8174589706374902,
#params {'max_depth': 6, 'learning_rate': 0.0896457098432712, 'n_estimators': 800, 'colsample_bytree': 0.5569921809317748, 'colsample_bylevel': 0.9143406631538383, 'colsample_bynode': 0.5234481596946249, 'reg_lambda': 0.17888515294488294, 'reg_alpha': 0.12450084866814962, 'subsample': 0.95, 'min_child_weight': 4, 'gamma': 0.7192010329138374}
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    cbrm_param = {
        "objective": "binary:logistic",
        "booster": 'gbtree',
        #'tree_method':'gpu_hist', 'predictor':'gpu_predictor', 'gpu_id': 0, # GPU 사용시
        "tree_method": 'gpu_hist', 'gpu_id': 0,  # CPU 사용시
        "verbosity": 0,
        "max_depth": trial.suggest_int("max_depth", 4, 13),
        "learning_rate": trial.suggest_uniform('learning_rate', 0.08, 0.3), #0.08 ,0.3
        'n_estimators': trial.suggest_int("n_estimators",100,2000 ,step=100),
        
    }

    # Generate model
   
    model_cbrm = xgb.XGBClassifier(**cbrm_param  
                        ,boost_from_average=False)

    model_cbrm = model_cbrm.fit(x_train, y_train, eval_set=[(x_valid, y_valid)],early_stopping_rounds=200)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    F1 = f1_score(y_valid, model_cbrm.predict(x_valid))
    return F1

#optuna_cbrm = optuna.create_study(direction='maximize', sampler=sampler)
#optuna_cbrm.optimize(objective, n_trials=20)
#'colsample_bytree': 0.6547660592421192, 
#'colsample_bylevel': 0.9039959208689977, 
#'colsample_bynode': 0.7147460892158192, 
#'reg_lambda': 0.04264358848236461, 
#'reg_alpha': 0.18749180329486492, 
#'subsample': 0.75, 'min_child_weight': 2,
# 'gamma': 0.7584294257259844

In [ ]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params
print('Best Trial: score {},\nparams {}'.format(cbrm_trial.value, cbrm_trial_params))



In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(temp, y):
  folds.append((train_idx,valid_idx))

In [ ]:
random.seed(10)
best_params=[]
y_sum=y.copy()
y_sum=y_sum.astype('float')
y_sum=0
for fold in range(5):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  x_train = temp.iloc[train_idx].values 
  x_valid = temp.iloc[valid_idx].values
  y_train = y.iloc[train_idx]
  y_valid = y.iloc[valid_idx]
  optuna_cbrm = optuna.create_study(direction='maximize', sampler=sampler)
  optuna_cbrm.optimize(objective, n_trials=3)
  cbrm_trial = optuna_cbrm.best_trial
  cbrm_trial_params = cbrm_trial.params
  best_params.append(cbrm_trial_params)
  clf = xgb.XGBClassifier(**cbrm_trial.params,use_missing=False,boost_from_average=False)
  clf.fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=60,verbose=True)
  val_proba=clf.predict_proba(x_valid)[:,1]
  y_sum= y_sum+val_proba


  print(f'================================================================================\n\n')

In [ ]:
y_sum

- catboost optuna

In [ ]:
temp[['COMPONENT_ARBITRARY' ,'YEAR']]=temp[['COMPONENT_ARBITRARY' ,'YEAR']].astype('str')

In [ ]:
from sklearn.metrics import log_loss
from catboost import CatBoostClassifier

In [ ]:
def objective(trial):
  param = {
      "random_state":42,
      'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
      'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
      "n_estimators":trial.suggest_int("n_estimators", 1000, 10000),
      "max_depth":trial.suggest_int("max_depth", 4, 16),
      'random_strength' :trial.suggest_int('random_strength', 0, 100),
      "colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
      "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "max_bin": trial.suggest_int("max_bin", 200, 500),
      'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
  }

  X_train, X_valid, y_train, y_valid = train_test_split(temp,y,test_size=0.2)
  
  cat_features =['COMPONENT_ARBITRARY' ,'YEAR']
  cat = CatBoostClassifier(**param)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=50,cat_features=cat_features,
          verbose=100)
  cat_pred = cat.predict_proba(X_valid)
  m_F1 = f1_score(y_valid, cat.predict(X_valid),average='macro')
  
  return m_F1

In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:",study.best_value)
print("Best trial",study.best_trial.params)

- catboost kfold

In [ ]:
import random

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(temp, y):
  folds.append((train_idx,valid_idx))

In [ ]:
random.seed(42)
cat_models={}

cat_features =[0,2]
cat_proba=np.zeros(shape=(14095,1))
for fold in range(5):
  print(f'===================================={fold+1}============================================')
  train_idx, valid_idx = folds[fold]
  X_train = temp.iloc[train_idx].values 
  X_valid = temp.iloc[valid_idx].values
  y_train = y[train_idx]
  y_valid = y[valid_idx]

  cat = CatBoostClassifier(**study.best_params)
  cat.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_valid,y_valid)],
          early_stopping_rounds=35,cat_features=cat_features,
          verbose=100)
  cat_proba=cat_proba+cat.predict_proba(temp)
  print(f'================================================================================\n\n')

In [ ]:
cat_proba=cat_proba/5.0

In [ ]:
sum(np.where(cat_proba > 0.3, True, False))

#Optuna_Tuning ( Second_Regression)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
x_r_train.dtypes

In [ ]:
# 데이터 분리에서 x_train , x_valid , y_train , y_valid 
#분리해놓은 것을 통해 최적 파라미터를 탐색한다.
#Best Trial: score 0.8174589706374902,
#params {'max_depth': 6, 'learning_rate': 0.0896457098432712, 'n_estimators': 800, 'colsample_bytree': 0.5569921809317748, 'colsample_bylevel': 0.9143406631538383, 'colsample_bynode': 0.5234481596946249, 'reg_lambda': 0.17888515294488294, 'reg_alpha': 0.12450084866814962, 'subsample': 0.95, 'min_child_weight': 4, 'gamma': 0.7192010329138374}
# random sampler

#Best Trial: score 0.8174589706374902,
#params {'max_depth': 6, 'learning_rate': 0.0896457098432712, 'n_estimators': 800, 'colsample_bytree': 0.5569921809317748, 'colsample_bylevel': 0.9143406631538383, 'colsample_bynode': 0.5234481596946249, 'reg_lambda': 0.17888515294488294, 'reg_alpha': 0.12450084866814962, 'subsample': 0.95, 'min_child_weight': 4, 'gamma': 0.7192010329138374}
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    cbrm_param = {
         # 회귀
        "objective": "reg:squarederror",
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000,step=100),
        "max_depth": trial.suggest_int("max_depth", 4, 10),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    # Generate model
   
    model_cbrm_reg = xgb.XGBRegressor(**cbrm_param  
                        ,boost_from_average=False)

    model_cbrm_reg = model_cbrm_reg.fit(x_r_train, y_r_train, eval_set=[(x_r_valid, y_r_valid)],early_stopping_rounds=200)
                           
	# 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    mae = mean_absolute_error(y_r_valid, model_cbrm_reg.predict(x_r_valid))
    return mae




In [ ]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params
print('Best Trial: score {},\nparams {}'.format(cbrm_trial.value, cbrm_trial_params))



#LGB Regressor.

In [ ]:

from optuna.samplers import TPESampler

sampler = TPESampler(seed=10)

def objective(trial):
   

    param = {
        'objective': 'regression', # 회귀
        
        'metric': 'mae', 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = lgb.LGBMRegressor(**param)
    lgb_model = model.fit(x_r_train, y_r_train, eval_set=[(x_r_valid, y_r_valid)], verbose=0, early_stopping_rounds=75)
    mae = mean_absolute_error(y_r_valid, lgb_model.predict(x_r_valid))
    return mae
        
study_lgb = optuna.create_study(direction='minimize', sampler=sampler)
study_lgb.optimize(objective, n_trials=20)

In [ ]:
trial = study_lgb.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

